<a href="https://colab.research.google.com/github/lilnoes/notebooks/blob/main/forex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

In [ ]:
def getData(filename, length, usecolsx=[], usecolsy=[], cols=[], names=[]):
    x_feats = len(usecolsx)
    y_feats = len(usecolsy)
    data = pd.read_csv(filename, usecols=cols, names=names, sep=sep)
    size, _ = data.shape
    outx = np.zeros((size-length, length, x_feats))
    outy = np.zeros((size-length, y_feats))
    
    for i in range(size-length):
        outx[i] = data.iloc[i:i+length, usecolsx].to_numpy()

        outy[i] = data.iloc[i+length, usecolsy].to_numpy() > data.iloc[i+length-1, usecolsy].to_numpy()
        
    return outx, outy

def getModel():
  inputs = keras.Input(shape=(length,x_feats))
  x = keras.layers.LSTM(rnn_units) (inputs)
  # x = keras.layers.LSTM(rnn_units, stateful=True) (x)
  # x = keras.layers.Dropout(rate)(x)
  x = keras.layers.Dense(dense_units1, activation='relu')(x)
  # x = keras.layers.Dense(dense_units2, activation='relu')(x)
  x = keras.layers.Dense(y_feats, activation='sigmoid')(x)
  return keras.Model(inputs=inputs, outputs=x)

class MySave(tf.keras.callbacks.Callback):
  def __init__(self, checkpoint, directory):
    super(MySave, self).__init__()
    self.manager = tf.train.CheckpointManager(checkpoint, directory, max_to_keep=1)
    self.on_train_begin(None, False)

  def on_train_begin(self, logs=None, l=True):
    d = 0
    if save is True:
      d = self.manager.restore_or_initialize()
    if d:
      print('Model restored')
      if l:
        print('prev acc = ', self.manager.checkpoint.val_acc)
    else:
      print('Model initialized')

  def on_epoch_end(self, epoch, logs):
    if logs.get('val_acc', 0.0) >= self.manager.checkpoint.val_acc:
      print('Saving model...')
      self.manager.checkpoint.val_acc.assign(logs.get('val_acc'))
      self.manager.save()

def getDatasets(filename, test_size=0.2):
  x, y = getData(filename, length, usecolsx, usecolsy, cols, names)
  train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=test_size)
  trainds = tf.data.Dataset.from_tensor_slices((train_x, train_y))
  trainds = trainds.shuffle(SHUFFLE).batch(BATCH).cache()
  for i in trainds:
    pass
  trainds = trainds.repeat()

  testds = tf.data.Dataset.from_tensor_slices((test_x, test_y))
  testds = testds.batch(BATCH)
  for i in testds:
    pass
  # return trainds, testds
  return trainds, testds, x, y


In [ ]:
def trainAndSave(filename):
  checkpoint_prefix = os.path.splitext(filename)[0]
  checkpoint_directory = os.path.join(ml_folder, "forex", "saved_models", checkpoint_prefix)
  data_filename = os.path.join(ml_folder, "forex", "datasets", filename)

  optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
  loss = keras.losses.BinaryCrossentropy()
  history = keras.callbacks.History()

  trainds, testds, _, _= getDatasets(data_filename)
  print(trainds, testds)

  model = getModel()

  model.compile(optimizer=optimizer, loss=loss)

  checkpoint = tf.train.Checkpoint(model=model, val_acc=tf.Variable(0.0))


  mySave = MySave(checkpoint, checkpoint_directory)

  mySave.manager.restore_or_initialize()

  hist = model.fit(trainds, epochs=epochs, validation_data=testds, steps_per_epoch=8, callbacks=[history, mySave])
  histories[checkpoint_prefix] = hist


  mySave.manager.restore_or_initialize()
  
  save_path = os.path.join(ml_folder, "forex", "saved_models", "lite", checkpoint_prefix+".lite")
  print(save_path)

  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  tflite_model = converter.convert()

  tf.io.write_file(save_path, tflite_model)

In [ ]:
ml_folder = os.path.join("/content/drive/Shared drives/lilnoes", "ml")
filenames_dir = os.path.join(ml_folder, "forex", "datasets")
filenames = os.listdir(filenames_dir)
print(filenames)
names = ['open', 'high', 'low', 'close', 'volume', 'rsi14', 'rsi20', 'wma50', 'wma100', 'ema',
        'obv', 'adxmain', 'adxplus', 'adxminus',
         'macdmain', 'macdsignal',
         'bandmain', 'bandupper', 'bandlower'
        ]
cols = list(range(1,20))
print(len(cols), len(names))

usecolsx = list(range(19))
x_feats = len(usecolsx)
sep = ';'

usecolsy = [3]
y_feats = len(usecolsy)
length = 240


BATCH = 64
SHUFFLE = 20000
rnn_units = 256
dense_units1 = 16
dense_units2 = 16
epochs = 10
rate = 0.25
learning_rate=1.0e-1

save = False

histories = {}



['EURUSDH1.csv', 'EURUSDH4.csv', 'EURUSDD1.csv', 'EURUSDM30.csv', 'EURUSDM15.csv']
19 19


In [ ]:
filename = "EURUSDM15.csv"
checkpoint_prefix = os.path.splitext(filename)[0]
checkpoint_directory = os.path.join(ml_folder, "forex", "saved_models", checkpoint_prefix)
data_filename = os.path.join(ml_folder, "forex", "datasets", filename)

optimizer = keras.optimizers.SGD(lr=learning_rate)
loss = keras.losses.BinaryCrossentropy()
history = keras.callbacks.History()

# trainds, testds, x, y= getDatasets(data_filename)
# print(trainds, testds)

model = getModel()

model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

checkpoint = tf.train.Checkpoint(model=model, val_acc=tf.Variable(0.0))

mySave = MySave(checkpoint, checkpoint_directory)

# mySave.manager.restore_or_initialize()

histories[checkpoint_prefix] = hist


# mySave.manager.restore_or_initialize()

# save_path = os.path.join(ml_folder, "forex", "saved_models", "lite", checkpoint_prefix+".lite")
# print(save_path)

# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()

# tf.io.write_file(save_path, tflite_model)

Model initialized


In [ ]:
model.fit(trainds, epochs=50, steps_per_epoch=10)

Epoch 1/50
10/10 [==============================] - 1s 69ms/step - loss: 0.6936 - acc: 0.4984
Epoch 2/50
10/10 [==============================] - 1s 57ms/step - loss: 0.6929 - acc: 0.5125
Epoch 3/50
10/10 [==============================] - 1s 57ms/step - loss: 0.6941 - acc: 0.4781
Epoch 4/50
10/10 [==============================] - 1s 57ms/step - loss: 0.6927 - acc: 0.5281
Epoch 5/50
10/10 [==============================] - 1s 57ms/step - loss: 0.6926 - acc: 0.5203
Epoch 6/50
10/10 [==============================] - 1s 58ms/step - loss: 0.6940 - acc: 0.4844
Epoch 7/50
10/10 [==============================] - 1s 58ms/step - loss: 0.6931 - acc: 0.5109
Epoch 8/50
10/10 [==============================] - 1s 57ms/step - loss: 0.6934 - acc: 0.4953
Epoch 9/50
10/10 [==============================] - 1s 58ms/step - loss: 0.6935 - acc: 0.4844
Epoch 10/50
10/10 [==============================] - 1s 57ms/step - loss: 0.6932 - acc: 0.4797
Epoch 11/50
10/10 [==============================] - 1s 56m

In [ ]:
num = 5
x[-(num-1),-1], x[-num, -1], y[-num]

(array([ 1.1781    ,  1.17834   ,  1.17742   ,  1.17783   ,  0.1725    ,
         0.31188937,  0.33320447,  1.18109425,  1.18365885,  1.18191238,
        -0.483688  ,  0.16195135,  0.14459912,  0.23871197, -0.00278471,
        -0.00236116,  1.182672  ,  1.19158165,  1.17376235]),
 array([ 1.17876   ,  1.17884   ,  1.17807   ,  1.1781    ,  0.1163    ,
         0.32394043,  0.34162027,  1.18129315,  1.18379909,  1.18207901,
        -0.481963  ,  0.15854014,  0.15050113,  0.21961769, -0.00275169,
        -0.00228815,  1.1829018 ,  1.19199256,  1.17381104]),
 array([0.]))

In [ ]:
num = 1
xx = np.expand_dims(x[-num], axis=0)
xx = xx.astype(dtype=np.float32)
preds = model.predict(xx)
preds, y[-num], checkpoint_directory

(array([[0.49711612]], dtype=float32),
 array([1.]),
 '/content/drive/Shared drives/lilnoes/ml/forex/saved_models/EURUSDM15')

In [ ]:
getOutput("EURUSDM15", xx)

array([[1.1726568]], dtype=float32)

In [ ]:
import tensorflow.lite as tflite
models = {}
MAIN_FOLDER = "/content/drive/Shared drives/lilnoes/ml/forex/saved_models/lite"
def getOutput(name, input_tensor):
	if name not in models:
		models[name] = {}
		models[name]['save_path'] = os.path.join(MAIN_FOLDER, name+".lite")
		models[name]['interpreter'] = tflite.Interpreter(models[name]['save_path'])
		models[name]['input_details'] = models[name]['interpreter'].get_input_details()
		models[name]['output_details'] = models[name]['interpreter'].get_output_details()
		models[name]['interpreter'].allocate_tensors()

	models[name]['interpreter'].set_tensor(models[name]['input_details'][0]['index'], input_tensor)
	models[name]['interpreter'].invoke()
	return models[name]['interpreter'].get_tensor(models[name]['output_details'][0]['index'])


In [ ]:
for filename in filenames:
  trainAndSave(filename)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
